In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LinearRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Configuração de exibição
pd.set_option('display.max_columns', None)
np.random.seed(42)

# Carregamento e Engenharia de Features
try:
    df_raw = pd.read_csv('dados_consumo_simulados(2).csv')
    print("Dados carregados com sucesso. Primeiras 5 linhas:")
    print(df_raw.head())

    # --- Pré-processamento e Engenharia de Features ---

    # 1. Limpeza/Cálculo de Quantidade Absoluta (Se 'quantidade' é o valor do consumo/estoque)
    # Assumimos que o problema se refere à gestão de estoque/consumo por Unidade e Material
    df_raw['quantidade_abs'] = df_raw['quantidade'].abs()

    # 2. Criação da Variável Alvo Contínua (Regressão)
    # Exemplo: Consumo Total Mínimo do Material
    df_agg = df_raw.groupby(['unidade', 'material']).agg(
        CONSUMO_REAL=('quantidade_abs', 'sum'),
        qtd_transacoes=('quantidade_abs', 'count'),
        media_transacao=('quantidade_abs', 'mean')
    ).reset_index()

    # 3. Criação da Variável Alvo Binária (Classificação)
    # Problema: Atraso na Entrega/Necessidade Crítica (ex: alto consumo).
    # Vamos simular o 'ATRASO_ENTREGA' para as unidades/materiais com consumo acima do 75º percentil.
    consumo_q75 = df_agg['CONSUMO_REAL'].quantile(0.75)
    df_agg['ATRASO_ENTREGA'] = (df_agg['CONSUMO_REAL'] > consumo_q75).astype(int)
    print(f"\nVariável 'ATRASO_ENTREGA' criada: 1 se CONSUMO_REAL > {consumo_q75:.2f} (Top 25% de Consumo).")
    print(df_agg.head())

    # 4. Preparação das Features para Modelagem (One-Hot Encoding)
    X = df_agg.drop(columns=['CONSUMO_REAL', 'ATRASO_ENTREGA'])
    y_class = df_agg['ATRASO_ENTREGA']
    y_reg = df_agg['CONSUMO_REAL']

    # Separando features categóricas e numéricas para pré-processamento
    categorical_features = ['unidade', 'material']
    numerical_features = ['qtd_transacoes', 'media_transacao']

    # Criação do Preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ],
        remainder='passthrough'
    )

    # Aplicação do Preprocessor e Divisão dos Dados
    X_processed = preprocessor.fit_transform(X)
    feature_names = numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features))
    X_df = pd.DataFrame(X_processed, columns=feature_names)

    # Divisão para Classificação
    X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
        X_df, y_class, test_size=0.3, random_state=42, stratify=y_class
    )
    # Divisão para Regressão (usamos os mesmos X_df e y_reg)
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X_df, y_reg, test_size=0.3, random_state=42
    )
    features = feature_names # Atualiza a lista de features para o código de modelagem

except FileNotFoundError:
    print("Erro: O arquivo 'dados_consumo_simulados(2).csv' não foi encontrado. Verifique o caminho.")
    # Adicione `return` se quiser parar a execução em caso de erro

In [ ]:
# Testando valores de K de 1 a 15
k_range = range(1, 16)
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_class, y_train_class, cv=5, scoring='accuracy')
    k_scores.append(scores.mean())

# O melhor K é o que maximiza a acurácia média da validação cruzada
best_k = k_range[np.argmax(k_scores)]
print(f"O melhor valor de K encontrado é: {best_k}")

# Modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=best_k)
knn_model.fit(X_train_class, y_train_class)
y_pred_knn = knn_model.predict(X_test_class)

In [ ]:
acc_knn = accuracy_score(y_test_class, y_pred_knn)
prec_knn = precision_score(y_test_class, y_pred_knn)
rec_knn = recall_score(y_test_class, y_pred_knn)
f1_knn = f1_score(y_test_class, y_pred_knn)

print("\n--- Métricas de Avaliação KNN ---")
print(f"Acurácia: {acc_knn:.4f}")
print(f"Precisão: {prec_knn:.4f}")
print(f"Recall: {rec_knn:.4f}")
print(f"F1-Score: {f1_knn:.4f}")
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_class, y_pred_knn))

In [ ]:
acc_knn = accuracy_score(y_test_class, y_pred_knn)
prec_knn = precision_score(y_test_class, y_pred_knn)
rec_knn = recall_score(y_test_class, y_pred_knn)
f1_knn = f1_score(y_test_class, y_pred_knn)

print("\n--- Métricas de Avaliação KNN ---")
print(f"Acurácia: {acc_knn:.4f}")
print(f"Precisão: {prec_knn:.4f}")
print(f"Recall: {rec_knn:.4f}")
print(f"F1-Score: {f1_knn:.4f}")
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test_class, y_pred_knn))

In [ ]:
# Modelo de Regressão Logística
logreg_model = LogisticRegression(solver='liblinear', random_state=42)
logreg_model.fit(X_train_class, y_train_class)
y_pred_logreg = logreg_model.predict(X_test_class)

# Interpretação dos Coeficientes
print("\n--- Interpretação dos Coeficientes da Regressão Logística ---")
coef_df = pd.DataFrame({
    'Feature': features,
    'Coeficiente': logreg_model.coef_[0],
    'Odds Ratio (exp(Coef))': np.exp(logreg_model.coef_[0])
})
coef_df = coef_df.sort_values(by='Odds Ratio (exp(Coef))', ascending=False).reset_index(drop=True)
print(coef_df.head(10)) # Exibe as 10 mais relevantes

# Interpretação
print("\nInterpretação:")
print("Features com Odds Ratio (OR) > 1 aumentam a chance de 'ATRASO_ENTREGA'.")
print("Features com OR < 1 diminuem a chance de 'ATRASO_ENTREGA'.")

In [ ]:
acc_logreg = accuracy_score(y_test_class, y_pred_logreg)
prec_logreg = precision_score(y_test_class, y_pred_logreg)
rec_logreg = recall_score(y_test_class, y_pred_logreg)
f1_logreg = f1_score(y_test_class, y_pred_logreg)

comparacao_class = pd.DataFrame({
    'Modelo': ['KNN', 'Regressão Logística'],
    'Acurácia': [acc_knn, acc_logreg],
    'F1-Score': [f1_knn, f1_logreg]
}).set_index('Modelo')

print("\n--- Comparação de Métricas de Classificação (KNN vs. LogReg) ---")
print(comparacao_class)

# Discussão
print("\nDiscussão - Vantagens:")
print("KNN: Bom para fronteiras complexas, mas 'caixa preta'.")
print("Regressão Logística: Oferece interpretabilidade via Odds Ratios, ideal para entender a influência de cada material/unidade no risco de atraso.")

In [ ]:
# Modelo de Regressão Linear Simples (Base)
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)

# Modelo Ridge (L2 Regularization)
alpha_ridge = 1.0 # Exemplo de Alpha
ridge_model = Ridge(alpha=alpha_ridge)
ridge_model.fit(X_train_reg, y_train_reg)

# Comparação dos Coeficientes
coef_comp_reg = pd.DataFrame({
    'Feature': features,
    'Linear_Reg_Coef': lin_reg.coef_,
    'Ridge_Coef': ridge_model.coef_
})

print(f"\n--- Comparação de Coeficientes: Regressão Linear vs. Ridge (Alpha={alpha_ridge}) ---")
print(coef_comp_reg.sort_values(by='Linear_Reg_Coef', ascending=False).head(5))

# Discussão
print("\nDiscussão - Efeito da Regularização Ridge:")
print("Ridge encolhe a magnitude dos coeficientes, mas não os zera. Isso estabiliza o modelo, prevenindo que features pouco importantes dominem, sem eliminá-las.")

In [ ]:
# Modelo Lasso (L1 Regularization)
alpha_lasso = 1.0 # Valor pode ser ajustado. Um valor maior força mais zeros.
lasso_model = Lasso(alpha=alpha_lasso, max_iter=10000)
lasso_model.fit(X_train_reg, y_train_reg)

# Comparação dos Coeficientes
coef_comp_reg['Lasso_Coef'] = lasso_model.coef_

print(f"\n--- Comparação de Coeficientes: Ridge vs. Lasso (Alpha={alpha_lasso}) ---")
print(coef_comp_reg.sort_values(by='Linear_Reg_Coef', ascending=False))

# Variáveis excluídas (coeficiente = 0)
excluded_vars_lasso = coef_comp_reg[coef_comp_reg['Lasso_Coef'].round(4) == 0]['Feature'].tolist()

# Discussão
print("\nDiscussão - Efeito da Regularização Lasso:")
print("Lasso zera coeficientes de features irrelevantes (seleção automática de variáveis).")
if excluded_vars_lasso:
    print(f"Variáveis excluídas (Coeficiente ~ 0) pelo Lasso: {len(excluded_vars_lasso)} variáveis.")
else:
    print("Nenhuma variável foi totalmente excluída com este alpha.")

In [ ]:
# Regressão Polinomial de Grau 2
degree = 2
poly = PolynomialFeatures(degree=degree, include_bias=False)
X_train_poly = poly.fit_transform(X_train_reg)
X_test_poly = poly.transform(X_test_reg)

# Modelo de Regressão Linear no novo conjunto
poly_reg = LinearRegression()
poly_reg.fit(X_train_poly, y_train_reg)
y_pred_poly = poly_reg.predict(X_test_poly)

# Métricas
y_pred_lin = lin_reg.predict(X_test_reg)
r2_lin = r2_score(y_test_reg, y_pred_lin)
r2_poly = r2_score(y_test_reg, y_pred_poly)

print(f"\n--- Comparação de Métricas de Regressão Contínua ---")
print(f"Regressão Linear Simples: R²={r2_lin:.4f}")
print(f"Regressão Polinomial (Grau {degree}): R²={r2_poly:.4f}")

# Avaliação
if r2_poly > r2_lin:
    print("Avaliação: Houve melhora. O modelo Polinomial capturou melhor as relações não-lineares.")
else:
    print("Avaliação: Não houve melhora significativa ou o desempenho piorou, sugerindo que as relações são predominantemente lineares.")

In [ ]:
# Função para avaliar Classificadores
def evaluate_classifier(y_test, y_pred, model_name):
    return {
        'Modelo': model_name,
        'Acurácia': accuracy_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred)
    }

# 1. Árvore de Decisão
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5)
dt_model.fit(X_train_class, y_train_class)
metrics_dt = evaluate_classifier(y_test_class, dt_model.predict(X_test_class), 'Árvore de Decisão')

# 2. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
rf_model.fit(X_train_class, y_train_class)
metrics_rf = evaluate_classifier(y_test_class, rf_model.predict(X_test_class), 'Random Forest')

comparacao_tree = pd.DataFrame([metrics_dt, metrics_rf]).set_index('Modelo')

print("\n--- Comparação de Desempenho: Árvore de Decisão vs. Random Forest ---")
print(comparacao_tree)

# Discussão
print("\nDiscussão - Interpretabilidade e Desempenho:")
print("Árvore de Decisão: ALTA interpretabilidade (regras claras).")
print("Random Forest: Geralmente melhor desempenho, pois reduz a variância (mais estável), mas é um modelo mais 'caixa preta'.")

In [ ]:
# Consolidação de Métricas de Classificação
all_class_metrics = pd.DataFrame([
    evaluate_classifier(y_test_class, y_pred_knn, 'KNN'),
    evaluate_classifier(y_test_class, y_pred_logreg, 'Regressão Logística'),
    metrics_dt, metrics_rf
])
all_class_metrics = all_class_metrics.sort_values(by='F1-Score', ascending=False).reset_index(drop=True)

print("\n\n--- Relatório Executivo - Comparação de Modelos ---")

print("\nModelos de Classificação (Atraso na Entrega - Y_Binário):")
print(all_class_metrics)

print("\nModelos de Regressão Contínua (Consumo Real - Y_Contínuo):")
reg_metrics = pd.DataFrame({
    'Modelo': ['Linear Simples', 'Ridge (L2)', 'Lasso (L1)', f'Polinomial (Grau {degree})'],
    'R²': [r2_lin, r2_score(y_test_reg, ridge_model.predict(X_test_reg)), r2_score(y_test_reg, lasso_model.predict(X_test_reg)), r2_poly],
})
reg_metrics = reg_metrics.sort_values(by='R²', ascending=False).reset_index(drop=True)
print(reg_metrics)

# Recomendação Final
best_class_model = all_class_metrics.iloc[0]

print("\n--- Recomendação Final ---")
print(f"Melhor Abordagem para o Desafio de Classificação (Atraso): **{best_class_model['Modelo']}**")
print("Justificativa:")
if best_class_model['Modelo'] == 'Regressão Logística':
    print("Recomendamos a Regressão Logística, pois, além do bom desempenho, ela oferece a **máxima interpretabilidade** para entender quais materiais/unidades mais contribuem para o risco de atraso (Odds Ratios). Isso é fundamental para a tomada de decisão operacional e intervenções.")
else:
    print(f"O **{best_class_model['Modelo']}** apresentou o melhor F1-Score/Acurácia. Ele é recomendado se o **desempenho preditivo bruto for a prioridade**, pois tende a ser mais robusto em dados complexos, mesmo que seja menos interpretável que a Regressão Logística.")